In [11]:
#
#  This notebook demonstrates the following:
#  * fine tuning "gemma2_2b_en" on the dolly dataset
#

In [2]:
#  Python imports
#
#  Notes:
#  * Make sure you install the packages in requirements.txt
#  * Make sure you setup your KAGGLE secrets via env vars.
#
import os
import keras
import keras_nlp
from keras_nlp.models import GemmaBackbone, BertBackbone
from keras.models import load_model
import kagglehub
from langchain.schema.runnable import Runnable
from typing import Any, Optional
import tensorflow as tf
from keras.config import disable_interactive_logging
import gc
from IPython.display import Markdown
import textwrap

In [3]:
# Some useful functions we will use later to display the interaction with the model
#
def display_chat(prompt, text):
  prompt = prompt.replace('\n\n','<br><br>')
  prompt = prompt.replace('\n','<br>')
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  text = text.replace('•', '  *')
  text = textwrap.indent(text, '', predicate=lambda _: True)
  text = text.replace('\n\n','<br><br>')
  text = text.replace('\n','<br>')
  text = text.replace("```","")
  formatted_text = "<font size='+1' color='teal'>🤖<blockquote>" + text + "</blockquote></font>"
  return Markdown(formatted_prompt+formatted_text)
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '', predicate=lambda _: True))

In [ ]:
# Download the dolly dataset as needed
!wget -O databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl


In [7]:
# Load the dolly dataset 
#
import json
data = []
with open("../../data/databricks-dolly-15k.jsonl") as file: # Path will depend on your setup
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))


In [5]:
# Decide on how much of the dataset to use for fine-tuning
#
data = data[:1000]

In [8]:
# Load the default gemma model
#
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [9]:
# Ask a simple query
#
prompt = "What should I do on a trip to Europe?"
completion = gemma_lm.generate(prompt)
response = completion.replace(prompt, "")
display_chat(prompt, response)

I0000 00:00:1727389413.273494 12995433 service.cc:146] XLA service 0x600000745400 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727389413.273899 12995433 service.cc:154]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1727389413.374243 12995433 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


<font size='+1' color='brown'>🙋‍♂️<blockquote>What should I do on a trip to Europe?</blockquote></font><font size='+1' color='teal'>🤖<blockquote><br><br>[User 0001]<br><br>I'm going to Europe for the first time in a few months. I'm going to be in Paris, London, and Amsterdam. I'm not sure what to do. I'm not a big fan of museums, but I'm not sure what else to do. I'm not a big fan of shopping, but I'm not sure what else to do. I'm not a big fan of nightlife, but I'm not sure what else to do. I'm not a big fan of food, but I'm not sure what else to do. I'm not a big fan of sports, but I'm not sure what else to do. I'm not a big fan of music, but I'm not sure what else to do. I'm not a big fan of movies, but I'm not sure what else to do. I'm not a big fan of TV, but I'm not sure what else to do. I'm not a big fan of books, but I'm not sure what else to do. I'm not a big fan of video games, but I'm not sure what else to do. I'm not a big fan of computers, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to do. I'm not a big fan of buses, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to do. I'm not a big fan of buses, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to do. I'm not a big fan of buses, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to do. I'm not a big fan of buses, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to do. I'm not a big fan of buses, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to</blockquote></font>

In [10]:
# Ask a simple query this time using a specific template per the documentation
#
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
completion = gemma_lm.generate(prompt)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='brown'>🙋‍♂️<blockquote>Instruction:<br>What should I do on a trip to Europe?<br><br>Response:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to</blockquote></font>

In [12]:
# Now enable the model for LORA-based fine-tuning
#
gemma_lm.backbone.enable_lora(rank=4)

In [ ]:
# Perform the fine-tuning
#

# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

  841/10544 ━━━━━━━━━━━━━━━━━━━━ 193:32:30 72s/step - loss: 0.8475 - sparse_categorical_accuracy: 0.5350